## Data Preprocessing

In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
%matplotlib inline

In [2]:
DIRECTORY=r'C:\Users\lucky\Miniproject_6th Sem\PetImages'
CATEGORIES=['cat','dog']
size=48

In [3]:
data=[]
for category in CATEGORIES:
    folder=os.path.join(DIRECTORY,category)
    label=CATEGORIES.index(category)   
    for img in os.listdir(folder):
        img_path=os.path.join(folder,img)
        image=plt.imread(img_path)
        try:
            R, G, B = image[:,:,0], image[:,:,1], image[:,:,2]
            image= 0.2989 * R + 0.5870 * G + 0.1140 * B
        except:
            image=image
        #plt.imshow(image)
        #break
        image=cv2.resize(image,(size,size))
        data.append([image,label])

C:\Users\lucky\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [4]:
len(data)

24997

In [5]:
random.shuffle(data)

In [6]:
data

[[array([[107.7988    , 108.5070625 , 106.7989    , ..., 119.90625156,
          125.67563984, 116.2416875 ],
         [108.9236875 , 108.15813906, 112.6719625 , ..., 119.9913125 ,
          120.15535859, 121.51585156],
         [110.7985    , 109.94181484, 109.33385234, ..., 119.1164    ,
          120.1163    , 119.1056    ],
         ...,
         [137.9449    , 143.55111016, 146.78027266, ..., 140.3751875 ,
          142.54424141, 140.81177578],
         [148.14440078, 138.64014297, 146.527375  , ..., 142.31808594,
          142.61493125, 140.8026125 ],
         [145.79905   , 146.65313125, 148.382125  , ..., 144.343925  ,
          142.65659375, 140.43980156]]),
  1],
 [array([[139.7672638 ,  69.41706875,  39.55457396, ...,  22.20715   ,
           21.812975  ,  24.8988    ],
         [ 97.46189453,  56.30591797,  34.50455885, ...,  22.20715   ,
           21.812975  ,  22.899     ],
         [ 76.48609583,  62.96114141,  28.74503516, ...,  23.3658    ,
           24.115725  ,  25

In [7]:
x=[]
y=[]
for features, labels in data:
    x.append(features)
    y.append(labels)

In [8]:
x[5].shape

(48, 48)

In [9]:
x=np.array(x)
y=np.array(y)

In [10]:
x.shape

(24997, 48, 48)

In [11]:
y.shape

(24997,)

In [12]:
pickle.dump(x,open('x.pkl','wb'))
pickle.dump(y,open('y.pkl','wb'))

## Training


In [13]:
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense   

In [14]:
x=pickle.load(open('x.pkl','rb'))
y=pickle.load(open('y.pkl','rb'))    

In [15]:
x=x/255

In [16]:
x.shape

(24997, 48, 48)

In [17]:
model=Sequential()
model.add(Conv2D(64,(3,3),input_shape=(size,size,1),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
#model.add(Conv2D(64,(3,3),activation='relu'))
#model.add(MaxPooling2D((2,2)))
model.add(Flatten())  

model.add(Dense(512,input_shape=x.shape[1:],activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


A CNN trains through forward propagation and back propagation for many epochs. This repeats until we have a well-defined neural network with trained weights and feature detectors.

In [18]:
model.fit(x,y,epochs=5,validation_split=0.15)   #, callbacks=[tensorboard]

Epoch 1/5
664/664 [==============================] - 464s 698ms/step - loss: 0.6208 - accuracy: 0.6464 - val_loss: 0.5513 - val_accuracy: 0.7165
Epoch 2/5
664/664 [==============================] - 501s 754ms/step - loss: 0.5077 - accuracy: 0.7524 - val_loss: 0.5102 - val_accuracy: 0.7411
Epoch 3/5
664/664 [==============================] - 485s 731ms/step - loss: 0.4472 - accuracy: 0.7893 - val_loss: 0.4892 - val_accuracy: 0.7557
Epoch 4/5
664/664 [==============================] - 520s 783ms/step - loss: 0.3884 - accuracy: 0.8227 - val_loss: 0.4886 - val_accuracy: 0.7741
Epoch 5/5
664/664 [==============================] - 496s 747ms/step - loss: 0.3015 - accuracy: 0.8687 - val_loss: 0.5007 - val_accuracy: 0.7725


## Prediction

In [19]:
def image(path):
    image = plt.imread(path)
    try:
        R, G, B = image[:,:,0], image[:,:,1], image[:,:,2]
        image= 0.2989 * R + 0.5870 * G + 0.1140 * B
    except:
        image=image
    new_arr = cv2.resize(image, (size, size))
    new_arr = np.array(new_arr)
    new_arr = new_arr.reshape(-1, size,size, 1)
    return new_arr

In [20]:
prediction = model.predict([image('testImages/d1.jpg')])
print(CATEGORIES[prediction.argmax()])

1/1 [==============================] - 0s 289ms/step
cat


In [21]:
prediction = model.predict([image('testImages/c4.jpg')])
print(CATEGORIES[prediction.argmax()])

1/1 [==============================] - 0s 51ms/step
cat


In [22]:
prediction = model.predict([image('testImages/d9.jpg')])
print(CATEGORIES[prediction.argmax()])

1/1 [==============================] - 0s 46ms/step
dog
